In [1]:
import re
import ast
import pyprind
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

from multiprocessing import Pool, Manager

import time
start = time.time()

In [2]:
url = r'https://www.cardmarket.com/de/Magic'
headers = {'User-agent': 'Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.76 Safari/537.36', 'Accept':'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8','Accept-Encoding':'gzip,deflate,sdch'}

In [3]:
agent = ['Mozilla/{}.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/29.0.1547.76 Safari/537.36'.format(i) for i in range(5, 10)]
accept = ['text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8']
encoding = ['gzip,deflate,sdch']

In [4]:
n = 5

manager = Manager()
cards = manager.list()

def pageGenerator(n):
    for i in range(n):
        yield i    

def getCardURLs(i, cards=cards):
    bs = BeautifulSoup(requests.get(url+'/Products/Singles?resultsPage={}'.format(i), headers=headers).text, 'lxml')
    cards.append([tr.findAll('a')[1]['href'] for tr in bs.findAll('tr') if tr.find('a')])

pool = Pool()
pool.map(getCardURLs, pageGenerator(n))

cards = {card for liste in cards for card in liste}
len(cards)

150

In [5]:
data = manager.list()
error_urls = manager.list()

def cardGenerator(cards):
    for i, card in enumerate(cards):
        if i%2==0:
            time.sleep(2)
        yield card

def getCardData(card, data=data, error_urls=error_urls):
    html = requests.get('https://www.cardmarket.com'+card, headers={'User-agent': np.random.choice(agent), 'Accept':np.random.choice(accept),'Accept-Encoding':np.random.choice(encoding)}).text
    bs = BeautifulSoup(html, 'lxml')
    try:
        chartData = ast.literal_eval(re.search('chartData =(.*?);', str(bs)).group(1))
        df = pd.DataFrame(list(zip(chartData['labels'], chartData['datasets'][0]['data'])), columns=['Date', 'Price'])
        df['Date'] = pd.to_datetime(df['Date'], dayfirst=True)
        df['Link'] = card
        splitted = card.split('/')
        df['Name'] = splitted[6]
        df['Edition'] = splitted[5]
        df['Rarity'] = bs.find('tbody').find('td', {'class':'outerTop outerRight col_Odd col_1 cell_0_1'}).find('span')['onmouseover'].split("'")[1]
        df['Available'] = int(bs.find('span', {'itemprop':'offerCount'}).get_text())
        data.append(df)
    except: return bs

pool = Pool(5)
_ = pool.map(getCardData, cardGenerator(cards))

In [6]:
df = pd.concat(data)
df = df.set_index('Date')
df.to_csv('build/MGT_Cards.csv')
print(df.shape)
df.head()

(4156, 6)


,Price,Link,Name,Edition,Rarity,Available
Date,,,,,,
2018-04-04,0.08,/de/Magic/Products/Singles/Dominaria/Entrei%C3...,Entrei%C3%9Fen,Dominaria,Common,3438
2018-04-05,0.06,/de/Magic/Products/Singles/Dominaria/Entrei%C3...,Entrei%C3%9Fen,Dominaria,Common,3438
2018-04-06,0.09,/de/Magic/Products/Singles/Dominaria/Entrei%C3...,Entrei%C3%9Fen,Dominaria,Common,3438
2018-04-07,0.06,/de/Magic/Products/Singles/Dominaria/Entrei%C3...,Entrei%C3%9Fen,Dominaria,Common,3438
2018-04-08,0.06,/de/Magic/Products/Singles/Dominaria/Entrei%C3...,Entrei%C3%9Fen,Dominaria,Common,3438


In [7]:
ende = time.time()
print('Duration: {}'.format(ende-start))

Duration: 194.62215423583984
